In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import math
import warnings
from scipy import stats
from scipy.stats import skew
from scipy.stats import kurtosis
from sklearn import linear_model
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import cohen_kappa_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split #we need the train_test split 
from sklearn.preprocessing import OneHotEncoder

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
%matplotlib inline




In [2]:
data = pd.read_csv('PRC_Data_Breach_Chronology_13012020(2).csv')
data.head(5)

#keywords for Description of Incident NLP (natural language processes-there are libraries)


,Date Made Public,Company,City,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,3/3/2006,PayDay OK LLC,NaN,New Jersey,HACK,BSF,88,The company's website was breached sometime ar...,California Attorney General,https://oag.ca.gov/,2006,40.058324,-74.405661,NaN,NaN,NaN
1,1/4/2012,"SF Fire Credit Union, Pacifica-Coastside Credi...",San Francisco,California,PORT,BSF,0,"The December 29, 2011 theft of a laptop from a...",California Attorney General,NaN,2012,37.774930,-122.419416,NaN,NaN,NaN
2,2/18/2012,"BDO USA, Rubio's Restaurants, Inc.",San Diego,California,PORT,BSR,0,BDO was contracted by Rubio's to perform finan...,California Attorney General,NaN,2012,32.715329,-117.157255,NaN,NaN,NaN
3,2/22/2012,"DHI Mortgage Company, Ltd.",Austin,Texas,HACK,BSF,0,"On February 10, 2012, DHI Mortgage became awar...",California Attorney General,NaN,2012,30.267153,-97.743061,NaN,NaN,NaN
4,3/12/2012,"Impairment Resources, LLC",San Diego,California,PORT,MED,"14,000",An office burglary on New Year's Eve 2011 resu...,California Attorney General,NaN,2012,32.715329,-117.157255,NaN,NaN,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9015 entries, 0 to 9014
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Date Made Public         9015 non-null   object 
 1   Company                  9015 non-null   object 
 2   City                     5690 non-null   object 
 3   State                    8436 non-null   object 
 4   Type of breach           8926 non-null   object 
 5   Type of organization     9015 non-null   object 
 6   Total Records            9009 non-null   object 
 7   Description of incident  9012 non-null   object 
 8   Information Source       8962 non-null   object 
 9   Source URL               3607 non-null   object 
 10  Year of Breach           9015 non-null   int64  
 11  Latitude                 6541 non-null   float64
 12  Longitude                6541 non-null   float64
 13  Unnamed: 13              0 non-null      float64
 14  Unnamed: 14             

In [4]:
data.columns=[e.lower().replace(' ', '_') for e in data.columns]  
data.columns

Index(['date_made_public', 'company', 'city', 'state', 'type_of_breach',
       'type_of_organization', 'total_records', 'description_of_incident',
       'information_source', 'source_url', 'year_of_breach', 'latitude',
       'longitude', 'unnamed:_13', 'unnamed:_14', 'unnamed:_15'],
      dtype='object')

In [5]:
import spacy
nlp = spacy.load('en_core_web_lg')


In [6]:
# testing function for tokenization on text

text = 'BDO was contracted, by Rubio'

def tokenize(text):
    return [tok.text for tok in nlp.tokenizer(text) if not tok.is_stop]

tokenize(text)

['BDO', 'contracted', ',', 'Rubio']

In [7]:
# go through all the cells create a big list with all descriptions combined and tokenized, select most repetitives token and than attribute ordinalization to the keywords 

In [8]:
#checking null in the column

data['description_of_incident'].isna().sum()

3

In [9]:
# printing the null rows to change

print(data[data["description_of_incident"].isnull()])

     date_made_public                     company         city       state  \
6373         2/5/2018  Charles Komar & Sons, Inc.  Jersey City  New Jersey   
6524        7/13/2015     Mule Creek State Prison         Ione  California   
6525        10/2/2015      Sentara Heart Hospital      Norfolk    Virginia   

     type_of_breach type_of_organization total_records  \
6373           UNKN                  BSO            99   
6524           DISC                  GOV             0   
6525           PORT                  MED         1,040   

     description_of_incident      information_source source_url  \
6373                     NaN  Security Breach Letter        NaN   
6524                     NaN                     NaN        NaN   
6525                     NaN                     NaN        NaN   

      year_of_breach   latitude   longitude  unnamed:_13  unnamed:_14  \
6373            2017  40.715202  -74.033868          NaN          NaN   
6524            2015  38.369629 -120.95

In [10]:
# filled the null values with unknown as otherwise the tokenization will not work 

data.at[6373,'description_of_incident']= 'Unknown,Unknown,Unknown,Unknown,Unknown'
data.at[6524,'description_of_incident']= 'Unknown,Unknown,Unknown,Unknown,Unknown'
data.at[6525,'description_of_incident']= 'Unknown,Unknown,Unknown,Unknown,Unknown'


In [11]:
# this one was not working before because of the null values!
def tokenize(text):
    return [tok.text for tok in nlp.tokenizer(text) if not tok.is_stop]

data['token'] = data['description_of_incident'].apply(lambda x: tokenize(x))

In [12]:
# now I have a new column for my token 

data['token'].head(20)

0     [company, website, breached, February, 19, hac...
1     [December, 29, ,, 2011, theft, laptop, parked,...
2     [BDO, contracted, Rubio, perform, financial, a...
3     [February, 10, ,, 2012, ,, DHI, Mortgage, awar...
4     [office, burglary, New, Year, Eve, 2011, resul...
5     [RJL, Insurance, Services, aware, vulnerabilit...
6     [purchased, hard, drive, September, 2011, imme...
7     [security, company, searching, web, sensitive,...
8     [hacker, able, access, names, ,, mailing, addr...
9     [storeroom, window, Saint, Joseph, HealthCare,...
10    [March, 6, ,, spreadsheet, containing, names, ...
11    [San, Francisco, Head, Start, /, Early, Head, ...
12    [Opening, Ceremony, discovered, inadvertent, b...
13    [March, 6, ,, 2012, ,, investigation, confirme...
14    [February, 15, ,, 2012, ,, Union, Bank, discov...
15    [hacker, hackers, able, intercept, customer, i...
16    [March, 23, ,, 2012, ,, X, -, Rite, learned, d...
17    [Computer, equipment, contained, patient, 

In [13]:
print(data['token'])

0       [company, website, breached, February, 19, hac...
1       [December, 29, ,, 2011, theft, laptop, parked,...
2       [BDO, contracted, Rubio, perform, financial, a...
3       [February, 10, ,, 2012, ,, DHI, Mortgage, awar...
4       [office, burglary, New, Year, Eve, 2011, resul...
                              ...                        
9010    [Location, breached, information, :, Theft, \n...
9011    [Location, breached, information, :, Loss, \n,...
9012    [Location, breached, information, :, Theft, \n...
9013    [Location, breached, information, :, Theft, \n...
9014    [Location, breached, information, :, Theft, \n...
Name: token, Length: 9015, dtype: object


In [14]:
from itertools import chain

text1=" ".join(str(i) for i in chain(*data['token']))
print(text1)

company website breached February 19 hacker attempt gain access certain customers ' private information . Social Security numbers , names , addresses , bank account names bank account numbers compromised . 88 individuals affected . December 29 , 2011 theft laptop parked car San Francisco resulted loss personal information .   information preparation merger SF Fire Credit Union Pacifica - Coastside Credit Union .   Current account holders names , Social Security numbers , dates birth , addresses , Pacifica - Coastside Credit Union account information . BDO contracted Rubio perform financial auditing services .   BDO employee accidentally removed CD - ROMs office .   CD - ROM CD - ROMs contained list Rubio workers ' compensation claimants list people owned equity shares Rubio Restaurants , Inc.   CD - ROM CD - ROMS appear stolen BDO employee vehicle .   workers ' compensation information contained names , claim numbers , medical status , date loss .   medical status information included 

In [15]:

def tokenize(text):
    return [tok.text for tok in nlp.tokenizer(text1.lower()) if not tok.is_stop]

tokenize(text1)

['company',
 'website',
 'breached',
 'february',
 '19',
 'hacker',
 'attempt',
 'gain',
 'access',
 'certain',
 'customers',
 "'",
 'private',
 'information',
 '.',
 'social',
 'security',
 'numbers',
 ',',
 'names',
 ',',
 'addresses',
 ',',
 'bank',
 'account',
 'names',
 'bank',
 'account',
 'numbers',
 'compromised',
 '.',
 '88',
 'individuals',
 'affected',
 '.',
 'december',
 '29',
 ',',
 '2011',
 'theft',
 'laptop',
 'parked',
 'car',
 'san',
 'francisco',
 'resulted',
 'loss',
 'personal',
 'information',
 '.',
 '\xa0 ',
 'information',
 'preparation',
 'merger',
 'sf',
 'fire',
 'credit',
 'union',
 'pacifica',
 '-',
 'coastside',
 'credit',
 'union',
 '.',
 '\xa0 ',
 'current',
 'account',
 'holders',
 'names',
 ',',
 'social',
 'security',
 'numbers',
 ',',
 'dates',
 'birth',
 ',',
 'addresses',
 ',',
 'pacifica',
 '-',
 'coastside',
 'credit',
 'union',
 'account',
 'information',
 '.',
 'bdo',
 'contracted',
 'rubio',
 'perform',
 'financial',
 'auditing',
 'services',
 

In [16]:
from collections import Counter
p = Counter(" ".join(tokenize(text1)).split()).most_common(3000)
rslt = pd.DataFrame(p, columns=['Word', 'Frequency'])
print(rslt)

             Word  Frequency
0               ,      22465
1               .      22176
2     information      13075
3               :       6803
4         numbers       5087
...           ...        ...
2995      stanley          7
2996    quarterly          7
2997    elections          7
2998         ipad          7
2999           s.          7

[3000 rows x 2 columns]


In [17]:
rslt['Word'].head(50)


0                ,
1                .
2      information
3                :
4          numbers
5         security
6           social
7            names
8                -
9         breached
10        business
11          breach
12        location
13         present
14       associate
15               "
16            data
17       addresses
18          access
19        employee
20               /
21           theft
22            card
23        personal
24          credit
25    unauthorized
26        affected
27               (
28          health
29           dates
30          stolen
31          number
32        computer
33           birth
34        included
35         exposed
36         records
37           email
38       contained
39          office
40        incident
41       employees
42          laptop
43         account
44         company
45       customers
46         patient
47         medical
48        services
49        patients
Name: Word, dtype: object

In [18]:
# dropping punctuations and words that maybe redundant or not meaningful 
most_words = rslt.drop([0,1,3,4,8,10,11,13,15,16,20,23,26,27,29,31,34,35,36,38,40,44,51,52,53,55,59,56,61,62,63,64,67,68,69,70,72,74,78,82,90,91,92,94], axis=0).head(3000)

In [19]:
most_words

,Word,Frequency
5,security,4641
6,social,3183
7,names,3167
9,breached,3087
12,location,2824
...,...,...
2995,stanley,7
2996,quarterly,7
2997,elections,7
2998,ipad,7


In [20]:
most_words = most_words.set_index("Word")


In [21]:
most_words

,Frequency
Word,
security,4641
social,3183
names,3167
breached,3087
location,2824
...,...
stanley,7
quarterly,7
elections,7


In [22]:
most_words = most_words.reset_index()


In [23]:
most_words

,Word,Frequency
0,security,4641
1,social,3183
2,names,3167
3,breached,3087
4,location,2824
...,...,...
2950,stanley,7
2951,quarterly,7
2952,elections,7
2953,ipad,7


In [24]:
list_of_words = most_words['Word'].to_list()


In [25]:
list_of_words

['security',
 'social',
 'names',
 'breached',
 'location',
 'associate',
 'addresses',
 'access',
 'employee',
 'theft',
 'card',
 'credit',
 'unauthorized',
 'health',
 'stolen',
 'computer',
 'birth',
 'email',
 'office',
 'employees',
 'laptop',
 'account',
 'customers',
 'patient',
 'medical',
 'services',
 'patients',
 'compromised',
 'disclosure',
 'attorney',
 'system',
 'customer',
 'website',
 'students',
 'online',
 'hackers',
 'bank',
 'phone',
 'university',
 'payment',
 'cards',
 'passwords',
 'files',
 'department',
 'accounts',
 'financial',
 'address',
 'indiana',
 'human',
 'insurance',
 'server',
 'documents',
 'tax',
 'identity',
 'drive',
 'current',
 'sensitive',
 'said',
 'new',
 'database',
 'student',
 'million',
 '1',
 'home',
 'hospital',
 'january',
 'center',
 'occurred',
 'july',
 'california',
 'party',
 'password',
 'fraudulent',
 'service',
 'hacker',
 '2011',
 'october',
 'debit',
 'june',
 'november',
 'march',
 'file',
 'april',
 'posted',
 'computer

In [26]:
data['token'].head(20)

0     [company, website, breached, February, 19, hac...
1     [December, 29, ,, 2011, theft, laptop, parked,...
2     [BDO, contracted, Rubio, perform, financial, a...
3     [February, 10, ,, 2012, ,, DHI, Mortgage, awar...
4     [office, burglary, New, Year, Eve, 2011, resul...
5     [RJL, Insurance, Services, aware, vulnerabilit...
6     [purchased, hard, drive, September, 2011, imme...
7     [security, company, searching, web, sensitive,...
8     [hacker, able, access, names, ,, mailing, addr...
9     [storeroom, window, Saint, Joseph, HealthCare,...
10    [March, 6, ,, spreadsheet, containing, names, ...
11    [San, Francisco, Head, Start, /, Early, Head, ...
12    [Opening, Ceremony, discovered, inadvertent, b...
13    [March, 6, ,, 2012, ,, investigation, confirme...
14    [February, 15, ,, 2012, ,, Union, Bank, discov...
15    [hacker, hackers, able, intercept, customer, i...
16    [March, 23, ,, 2012, ,, X, -, Rite, learned, d...
17    [Computer, equipment, contained, patient, 

In [27]:
import string
for n in range(len(data['token'])):
    str_list = [word for word in data['token'][n] if word not in string.punctuation]
    str_list = [word for word in str_list if word != '\xa0']
    str_list = [word for word in str_list if len(word) > 2]
    str_list = [word.lower() for word in str_list]
   

    data['token'][n]=str_list
    

In [28]:
data.token.head(20)

0     [company, website, breached, february, hacker,...
1     [december, 2011, theft, laptop, parked, car, s...
2     [bdo, contracted, rubio, perform, financial, a...
3     [february, 2012, dhi, mortgage, aware, softwar...
4     [office, burglary, new, year, eve, 2011, resul...
5     [rjl, insurance, services, aware, vulnerabilit...
6     [purchased, hard, drive, september, 2011, imme...
7     [security, company, searching, web, sensitive,...
8     [hacker, able, access, names, mailing, address...
9     [storeroom, window, saint, joseph, healthcare,...
10    [march, spreadsheet, containing, names, contac...
11    [san, francisco, head, start, early, head, sta...
12    [opening, ceremony, discovered, inadvertent, b...
13    [march, 2012, investigation, confirmed, employ...
14    [february, 2012, union, bank, discovered, cont...
15    [hacker, hackers, able, intercept, customer, i...
16    [march, 2012, rite, learned, database, server,...
17    [computer, equipment, contained, patient, 

In [29]:
rows_of_df = data['token'].tolist()

In [30]:
[x.lower() for x in rows_of_df[0]]

['company',
 'website',
 'breached',
 'february',
 'hacker',
 'attempt',
 'gain',
 'access',
 'certain',
 'customers',
 'private',
 'information',
 'social',
 'security',
 'numbers',
 'names',
 'addresses',
 'bank',
 'account',
 'names',
 'bank',
 'account',
 'numbers',
 'compromised',
 'individuals',
 'affected']

In [31]:
# LIST INTERSECTION!!!! 
# https://stackoverflow.com/questions/3697432/how-to-find-list-intersection _auxset

# a = [1,2,3,4,5]
# b = [1,3,5,6]
# output: [1,3,5]


#Make a set out of the larger one:

#_auxset = set(a)

# Save
#Then,

# [value for value in b if value in a]
#c = [x for x in b if x in _auxset]

_auxset = set([x.lower() for x in rows_of_df[0]])

In [32]:
c = [x for x in list_of_words if x in _auxset]

In [33]:
print(c)


['security', 'social', 'names', 'breached', 'addresses', 'access', 'account', 'customers', 'compromised', 'website', 'bank', 'hacker', 'february', 'certain', 'private', 'gain', 'attempt']


In [34]:
# Pick the first 5 

print(c[0:5])


['security', 'social', 'names', 'breached', 'addresses']


In [35]:
x = [word for word in list_of_words if word in data['token'][0]]
x

['security',
 'social',
 'names',
 'breached',
 'addresses',
 'access',
 'account',
 'customers',
 'compromised',
 'website',
 'bank',
 'hacker',
 'february',
 'certain',
 'private',
 'gain',
 'attempt']

In [36]:
#sorted(zip(x,list_of_words))

In [37]:
#[word for word in [tupple[0] for tupple in Counter(data['token'][0]).most_common(5)] if word in list_of_words]

In [49]:
data['token2'] = list(map(lambda x:  [word for word in list_of_words if word in x], data['token'] ))

In [70]:
data[['token','token2']].head(10)

,token,token2
0,"[company, website, breached, february, hacker,...","[security, social, names, breached, addresses,..."
1,"[december, 2011, theft, laptop, parked, car, s...","[security, social, names, addresses, theft, cr..."
2,"[bdo, contracted, rubio, perform, financial, a...","[security, social, names, employee, stolen, of..."
3,"[february, 2012, dhi, mortgage, aware, softwar...","[security, social, names, birth, system, onlin..."
4,"[office, burglary, new, year, eve, 2011, resul...","[security, social, names, addresses, office, m..."
5,"[rjl, insurance, services, aware, vulnerabilit...","[security, social, names, computer, medical, s..."
6,"[purchased, hard, drive, september, 2011, imme...","[security, social, names, addresses, employee,..."
7,"[security, company, searching, web, sensitive,...","[security, social, names, employee, birth, emp..."
8,"[hacker, able, access, names, mailing, address...","[names, addresses, access, card, credit, birth..."
9,"[storeroom, window, saint, joseph, healthcare,...","[security, social, names, addresses, services,..."


In [55]:
data.loc[data.make == '', 'make'] = data.token2.str.split().str.get(0)


AttributeError: Can only use .str accessor with string values!

In [56]:
data['make'] = data['token'].str.split(' ').str[0]


AttributeError: Can only use .str accessor with string values!

In [57]:
data.make = data.make.replace("", test.token2)


AttributeError: 'DataFrame' object has no attribute 'make'

In [45]:
token_dataframe = pd.DataFrame(data.token2)


In [46]:
token_dataframe

,token2
0,"[security, social, names, breached, addresses,..."
1,"[security, social, names, addresses, theft, cr..."
2,"[security, social, names, employee, stolen, of..."
3,"[security, social, names, birth, system, onlin..."
4,"[security, social, names, addresses, office, m..."
...,...
9010,"[breached, location, associate, theft]"
9011,"[breached, location, associate, loss]"
9012,"[breached, location, associate, theft]"
9013,"[breached, location, associate, theft]"


In [41]:
data['token2'].value_counts(sorted).reset_index()

,index,token2
0,"[breached, location, associate, theft]",0.094065
1,"[breached, location, associate, access, unauth...",0.080532
2,"[breached, location, associate]",0.066223
3,"[security, office, attorney, indiana]",0.055352
4,"[breached, location, associate, loss]",0.017526
...,...,...
5784,"[security, social, names, addresses, stolen, c...",0.000111
5785,"[addresses, card, credit, email, compromised, ...",0.000111
5786,"[security, social, names, students, university...",0.000111
5787,"[addresses, email, hackers, phone, accounts, h...",0.000111


In [42]:
data['token2']


0       [security, social, names, breached, addresses,...
1       [security, social, names, addresses, theft, cr...
2       [security, social, names, employee, stolen, of...
3       [security, social, names, birth, system, onlin...
4       [security, social, names, addresses, office, m...
                              ...                        
9010               [breached, location, associate, theft]
9011                [breached, location, associate, loss]
9012               [breached, location, associate, theft]
9013               [breached, location, associate, theft]
9014               [breached, location, associate, theft]
Name: token2, Length: 9015, dtype: object

In [43]:
data[['a','b','c','d','e']] = data['token2'].str.split(n=4, expand=True)


ValueError: Columns must be same length as key

In [ ]:
data[['a','b']] = data['token2'].str.split(n=1, expand=True)


In [ ]:
data.drop(['unnamed:_13', 'unnamed:_14','unnamed:_15','description_of_incident','source_url'], axis=1, inplace= True)

In [ ]:
data.info()

In [ ]:
n = data.nunique(axis=0) 
  
print("No.of.unique values in each column :\n", 
      n)

In [ ]:
data.isnull().sum()


In [ ]:
data.head(50)

#my target is type of breach 


fatturato
zone geografiche
piattaforme
tipo di frodi


In [ ]:
#used this to inspect the 39 rows 
duplicateRowsDF = data[data.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

In [ ]:
duplicateRowsDF.info()

In [ ]:
#there were 8 rows duplicates now removed
data=data.drop_duplicates()


In [ ]:
data.info()

In [ ]:
nulls = pd.DataFrame(data.isna().sum()/len(data))
nulls= nulls.reset_index()

In [ ]:
nulls.columns = ['column_name', 'Percentage Null Values']
nulls.sort_values(by='Percentage Null Values', ascending = False)

Data to fix with null values:

More than 25%:

2	city	0.368830

9	latitude	0.274432

10	longitude	0.274432

Less than 25% but still to fix:

3	state	0.064226

4	type_of_breach	0.009872

7	information_source	0.005879

6	total_records	0.000666

In [ ]:
data['state'].unique()

In [ ]:
def clean(x):    
    if x in ['Washington', 'WA']:
        return 'WA'
    elif x in ['Arizona','AZ']:
        return 'AZ'
    elif x in ['California','CA']:
        return 'CA'
    elif x in ['Nevada','NV']:
        return 'NV'
    elif x in ['Oregon','OR']:
        return 'OR'
    elif x in ['New Jersey','NJ']:
        return 'NJ'
    elif x in ['New York','NY']:
        return 'NY'
    elif x in ['Texas','TX']:
        return 'TX'
    elif x in ['Illinois','IL']:
        return 'IL'
    elif x in ['Michigan','MI']:
        return 'MI'
    elif x in ['Connecticut','CT']:
        return 'CT'
    elif x in ['Delaware','DE']:
        return 'DE'
    elif x in ['Maryland','MD']:
        return 'MD'
    elif x in ['Georgia','GA']:
        return 'GA'
    elif x in ['Florida','FL']:
        return 'FL'
    elif x in ['Ohio','OH']:
        return 'OH'
    elif x in ['Nebraska','IL']:
        return 'IL'
    elif x in ['Minnesota','MN']:
        return 'MN'
    elif x in ['Massachusetts','MA']:
        return 'MA'
    elif x in ['Colorado','CO']:
        return 'CO'
    elif x in ['Kentucky','KY']:
        return 'KY'
    elif x in ['District Of Columbia','DC']:
        return 'DC'
    elif x in ['Nevada','NV']:
        return 'NV'
    elif x in ['North Carolina','NC']:
        return 'NC'
    elif x in ['Missouri','MO']:
        return 'MO'
    elif x in ['Indiana','IN']:
        return 'IN'
    elif x in ['Alaska','AK']:
        return 'AK'
    elif x in ['Pennsylvania','PA']:
        return 'PA'
    elif x in ['Virginia','VA']:
        return 'VA'
    elif x in ['New Mexico','NM']:
        return 'NM'
    elif x in ['Hawaii','HI']:
        return 'HI'
    elif x in ['Tennessee','TN']:
        return 'TN'
    elif x in ['Vermont','VT']:
        return 'VT'
    elif x in ['Utah','UT']:
        return 'UT'
    elif x in ['Kansas','KS']:
        return 'KS'
    elif x in ['Alabama','AL']:
        return 'AL'
    elif x in ['Ontario','ON']:
        return 'ON'
    elif x in ['Iowa','IA']:
        return 'IA'
    elif x in ['South Carolina','SC']:
        return 'SC'
    elif x in ['Wisconsin','WI']:
        return 'WI'
    elif x in ['Quebec','QC']:
        return 'QC'
    elif x in ['Louisiana','LA']:
        return 'LA'
    elif x in ['Wyoming','WY']:
        return 'WY'
    elif x in ['New Hampshire','NH']:
        return 'NH'
    elif x in ['Oklahoma','OK']:
        return 'OK'
    elif x in ['Mississippi','MS']:
        return 'MS'
    elif x in ['Arkansas','AR']:
        return 'AR'
    elif x in ['Rhode Island','RI']:
        return 'RI'
    elif x in ['Maine','ME']:
        return 'ME'
    elif x in ['Montana','MT']:
        return 'MT'
    elif x in ['Idaho','ID']:
        return 'ID'
    elif x in ['West Virginia','WV']:
        return 'WV'
    elif x in ['South Dakota','SD']:
        return 'SD'
    elif x in ['North Dakota','ND']:
        return 'ND'
    elif x in ['Puerto Rico','PR']:
        return 'PR'
    elif x in ['Grand Bahama']:
        return 'Bahamas'
    elif x in ['Buckinghamshire ','Cheshire']:
        return 'UK'
    elif x in ['British Columbia']:
        return 'Canada'
    elif x in ['Noord Holland']:
        return 'Netherlands '
    elif x in ['Tokyo']:
        return 'Japan'
    elif x in ['Beijing','Guangdong']:
        return 'China'
    elif x in ['Berlin']:
        return 'Germany'
    elif x in ['Dublin']:
        return 'Ireland'
   


In [ ]:
data['state'] = list(map(clean, data['state']))

In [ ]:
data['state'].value_counts(sorted).reset_index()

In [ ]:
data['state'].isna().sum()

In [ ]:
state = data.dropna(subset=['state']).drop_duplicates('city').set_index('city')['state']
data['state'] = data['state'].fillna(data['city'].map(state))

In [ ]:
data['state'].isna().sum()

In [ ]:
data2 = data[data.isna().any(axis=1)]
print (data2)

In [ ]:
data['state'].isna().sum()

In [ ]:
data['type_of_breach'].value_counts(sorted).reset_index()

In [ ]:
data['type_of_breach'].isna().sum()

In [ ]:
most = data['type_of_breach'].value_counts(sorted).reset_index().iloc[0,0]

In [ ]:
data['type_of_breach'] = data['type_of_breach'].fillna(most)

In [ ]:
data['type_of_breach'].isna().sum()

In [ ]:
data.isnull().sum()


In [ ]:
data['information_source'].value_counts(sorted).reset_index()

In [ ]:
data["information_source"].fillna("Unknown", inplace = True) 


In [ ]:
data['information_source'].value_counts(sorted).reset_index()

In [ ]:
data.isnull().sum()


In [ ]:
data['total_records'].value_counts(sorted).reset_index()

In [ ]:
data['total_records'].unique

In [ ]:
data['total_records'] = data['total_records'].str.replace(",","").astype(float)


In [ ]:
data['total_records'].unique

In [ ]:
#data['total_records']=np.log(data['total_records']+1)

In [ ]:
#sns.distplot(data[data['total_records'].isna()==False]['total_records']) 
#plt.show()


In [ ]:
data['total_records'] = data['total_records'].fillna(np.mean(data['total_records']))

In [ ]:
data.isnull().sum()


In [ ]:
city = data.dropna(subset=['city']).drop_duplicates('state').set_index('state')['city']
data['city'] = data['city'].fillna(data['state'].map(city))

In [ ]:
data.isnull().sum()


In [ ]:
data['city'].unique()

In [ ]:
data['city'].value_counts(sorted).reset_index()

In [ ]:
latitude = data.dropna(subset=['latitude']).drop_duplicates('city').set_index('city')['latitude']
data['latitude'] = data['latitude'].fillna(data['city'].map(latitude))

In [ ]:
data.isnull().sum()


In [ ]:
longitude = data.dropna(subset=['longitude']).drop_duplicates('city').set_index('city')['longitude']
data['longitude'] = data['longitude'].fillna(data['city'].map(longitude))

In [ ]:
data.isnull().sum()


In [ ]:
state1 = data.dropna(subset=['state']).drop_duplicates('latitude').set_index('latitude')['state']
data['state'] = data['state'].fillna(data['latitude'].map(state1))

In [ ]:
data.isnull().sum()


In [ ]:
#  https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6 was an alternative solution for geocode 

In [ ]:
data.info()


In [ ]:
data.at[175,'state']= 'Unknown'
data.at[342,'state']= 'CA'
data.at[521,'city']= 'Nassau'
data.at[601,'city']= 'London'
data.at[4342,'state']= 'UK'
data.at[4901,'state']= 'Germany'
data.at[5237,'city']= 'Tokyo'
data.at[5258,'state']= 'Unknown'
data.at[5435,'state']= 'NY'
data.at[5633,'city']= 'Dublin'
data.at[5664,'state']= 'China'
data.at[6077,'state']= 'MA'
data.at[6077,'city']= 'Boston'


In [ ]:
data.info()

In [ ]:
# The column City has 1500 unique values and instead of discarding it as it would influence the dummy process we will make it ordinal 

from sklearn.preprocessing import LabelEncoder

# Creating an instance of labelencoder
labelencoder = LabelEncoder()

# Assigning numerical values and storing it in another column called "City_N" 
data['city_n'] = labelencoder.fit_transform(data['city'])

# display Dataframe
data 

### Change date_made_public column to datetime format.


In [ ]:
data['date_made_public'] = pd.to_datetime(data.date_made_public)


In [ ]:
data['date_made_public']

In [ ]:
# qcut to bucket 'other' ask to Ignacio if you do not remember because otherwise
# what might happen is that we provide too much weight to other as a class 

In [ ]:
cat = data.select_dtypes('object')
num = data.select_dtypes(np.number)

In [ ]:
for i in cat:
    print(str(i) + ' ' + str(len(data[i].unique())))

In [ ]:
for i in num:
    print(str(i) + ' ' + str(len(data[i].unique())))

In [ ]:
data = data.set_index("company")


In [ ]:
data.head()

In [ ]:
from itertools import combinations
from scipy.stats import chi2_contingency



def data_corr_mat(df,confidence):
    columns = df.columns
    chi = pd.DataFrame(index=columns, columns=columns)
    p = pd.DataFrame(index=columns, columns=columns)
    nh = pd.DataFrame(index=columns, columns=columns)
    dof = pd.DataFrame(index=columns, columns=columns)
    for var1, var2 in combinations(columns, 2):
        cont_table = pd.crosstab(df[var1], df[var2], margins=False)
        chi2_stat= chi2_contingency(cont_table)[0]
        p_stat= chi2_contingency(cont_table)[1]
        chi.loc[var2, var1] = chi2_stat
        chi.loc[var1, var2] = chi2_stat
        p.loc[var2, var1] = p_stat
        p.loc[var1, var2] = p_stat
        dof_stat= chi2_contingency(cont_table)[2]
        dof.loc[var2, var1] = dof_stat
        dof.loc[var1, var2] = dof_stat
        if p.loc[var2, var1] < confidence:
            nh.loc[var2, var1]='rejected'
        else:
            nh.loc[var2, var1]='accepted'
        if p.loc[var1, var2] < confidence:
            nh.loc[var1, var2]='rejected'
        else:
            nh.loc[var1, var2]='accepted'   
    chi.fillna(0, inplace=True)
    p.fillna(0, inplace=True)
    return chi,p,nh,dof

chi,p,nh,dof=data_corr_mat(cat,.05) 

In [ ]:
nh

In [ ]:
p

In [ ]:
corr_matrix=num.corr(method='pearson')
fig, ax = plt.subplots(figsize=(10, 8))
ax = sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
# longitude, year of breach

In [ ]:
data.columns

In [ ]:
data['type_of_organization'].value_counts(dropna=False)  


In [ ]:
data['year_of_breach'].value_counts(dropna=False)  


In [ ]:
cat

In [ ]:
num

In [ ]:
#making a copy of the original dataframe 
data_copy=data.copy()
data_copy.head()

In [ ]:
data_copy['total_records']=np.log(data_copy['total_records']+1)

In [ ]:
sns.distplot(data_copy[data_copy['total_records'].isna()==False]['total_records']) 
plt.show()

In [ ]:
plt.figure(figsize=(16,8))

sns.lineplot(x = 'information_source', y = 'date_made_public', data = data_copy) 
plt.xticks(rotation=60)
plt.show

In [ ]:
plt.figure(figsize=(16,8))

sns.lineplot(x = 'state', y = 'date_made_public', data = data_copy) 
plt.xticks(rotation=60)
plt.show

In [ ]:
#this means that there is no discrepancy between year of breach and date made public, we will use in our model year of breach but not the 2 of them

sns.lineplot(x = 'year_of_breach', y = 'date_made_public', data = data_copy) 
plt.xticks(rotation=60)
plt.show

In [ ]:

for i in data_copy.columns:
    print(str(i) + ' ' + str(len(data_copy[i].unique())))

In [ ]:
data_copy.drop('date_made_public',axis='columns', inplace=True)

In [ ]:
# dropping city as we have already ordinalized before the column  with LabelEncoder 

data_copy.drop('city',axis='columns', inplace=True)

In [ ]:
data_copy

In [ ]:
cat = data_copy.select_dtypes('object')
num = data_copy.select_dtypes(np.number)

In [ ]:
cat

In [ ]:
num

In [ ]:
data_copy=pd.get_dummies(data_copy,drop_first=True)
print("data encoded with dummies!") 

In [ ]:
data_copy

In [ ]:
data_copy.info()

In [ ]:
X_prep = StandardScaler().fit_transform(data_copy)


In [ ]:
X_prep

In [ ]:
import pickle

scaler = StandardScaler().fit(X_prep)
pickle.dump(scaler, open('scaler.pickle', 'wb'))

In [ ]:
pd.DataFrame(X_prep, columns=data_copy.columns).head(20)

In [ ]:

correlations_matrix = data_copy.corr()
mask = np.zeros_like(correlations_matrix)
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(50, 30))
ax = sns.heatmap(correlations_matrix, mask=mask, annot=True)
plt.show()

In [ ]:
#Preparing kmeans for our first model 
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=72, random_state=1234)
kmeans.fit(X_prep)

In [ ]:
import joblib 

In [ ]:
#storing the model 

joblib.dump(kmeans, 'model.pkl')  
model_loaded = joblib.load('model.pkl')

In [ ]:
#checking number of values per cluster 
clusters = kmeans.predict(X_prep)
pd.Series(clusters).value_counts().sort_index()

In [ ]:
#integrated our dataframe with the clusters

X_df = pd.DataFrame(X_prep)
X_df["cluster"] = clusters
X_df.head(50)

In [ ]:
kmeans.inertia_

In [ ]:
K = range(50, 100)
inertia = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(X_prep)
    inertia.append(kmeans.inertia_)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Elbow Method showing the optimal k')

In [ ]:
#from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_score

K = range(20, 100)
silhouette = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(X_prep)
    silhouette.append(silhouette_score(X_prep, kmeans.predict(X_prep)))


plt.figure(figsize=(20,8))
plt.plot(K, silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Silhouette Method showing the optimal k')

In [ ]:
kmeans = KMeans(n_clusters=72,
                init="random",
                n_init=10,  # try with 1, 4, 8, 20, 30, 100...
                max_iter=300,
                tol=0,
                algorithm="full",
                random_state=0)
kmeans.fit(X_prep)
print(kmeans.inertia_)

In [ ]:
wcss = []
for i in range(60,100):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(X_prep)
    wcss.append(kmeans.inertia_)
plt.plot(range(60,100), wcss, 'o')
plt.plot(range(60 , 100) , wcss , '-' , alpha = 0.5)
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.savefig('Elbow_Method.png')
plt.show()

In [ ]:
wcss

In [ ]:
kmeans = KMeans(n_clusters = 72, init = 'k-means++', max_iter = 400, n_init = 100, random_state = 0)
y_kmeans = kmeans.fit_predict(X_prep)

In [ ]:
y_kmeans.shape

In [ ]:
X_prep.shape

In [ ]:
kmeans = KMeans(n_clusters=72)    
kmeans.fit(X_prep)
y_kmeans = kmeans.predict(X_prep)
y_kmeans

In [ ]:
data["cluster"]=clusters
data.head(30)

In [ ]:
data.to_csv(r'Cluster_data1.csv', index = False)


In [ ]:
data = data.reset_index()

In [ ]:
data

In [ ]:
#data.to_csv(r'Cluster_data3.csv', index = False)


In [ ]:
data[data['cluster']==1].values